<a href="https://colab.research.google.com/github/IKKEM-Lin/colab/blob/main/path_search_20230901.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 产物路径搜索，需上传[ttl文件](https://github.com/IKKEM-Lin/colab/blob/main/gen_turtle_20230901.ipynb)

In [ ]:
# install dependencies
! pip install pubchempy
! pip install rdflib
! pip install requests
! pip install loguru
! pip install networkx

import pubchempy as pcp
from rdflib import Namespace, Literal, URIRef, Graph as RDFGraph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
from networkx import Graph as NXGraph
import requests
from loguru import logger

import copy
import json
import uuid
import collections
import hashlib
import os
import re

In [ ]:
rg = RDFGraph()
rg.parse("./tripple.ttl", format='turtle')

G = rdflib_to_networkx_multidigraph(rg)
print("networkx Graph loaded successfully with length {}".format(len(G)))
# Density
print("DENSITY")
print("============")
print("The network density is {}".format(nx.density(G)))

In [ ]:
edges = G.edges
def find_edge(produce, property, edges = edges):
  result = filter(lambda x: x[1] == produce and x[2] == property, edges)
  return result

def find_reactions_from_product(produce):
  result = find_edge(produce, URIRef('react:has_product'))
  return map(lambda x: x[0], result)

def find_reactants_from_reaction(reaction):
  result = find_edge(reaction, URIRef('spi:is_reactant_of'))
  return map(lambda x: x[0], result)

def get_products_from_reaction(reaction):
  result = find_edge(reaction, URIRef('spi:is_product_of'))
  return map(lambda x: x[0], result)

def is_reaction(node):
  result = filter(lambda x: x[0] == node and x[2] == URIRef('react:id'), edges)
  return True if list(result) else False

def search_one(name, max_steps=3):
  def get_spieces(name):
    # 拿到name对应的ID
    id_str, URI = get_spieces_class_operations(name, {})
    if "CHEBI" in id_str:
      return URIRef("obo:{}".format(id_str))
    else:
      return URIRef("spi:{}".format(id_str))
  query_str = get_spieces(name)
  duplicated_reactions = list(find_reactions_from_product(query_str));
  res_path = list(map(lambda x: [query_str, x], find_reactions_from_product(query_str)))
  logger.info(query_str) #################
  result_hash = ""
  condition = lambda path: len(path) < max_steps*2
  while any(map(condition ,res_path)):
    new_res_paths = []
    wait_process_paths = []
    for path in res_path:
      if condition(path):
        wait_process_paths.append(path)
      else:
        new_res_paths.append(path)
    for path in wait_process_paths:
      # print(path, is_reaction(path[-1])) #################
      if is_reaction(path[-1]):
        temp_reactants = list(find_reactants_from_reaction(path[-1]))
        substant_in_previous = any([item in path for item in temp_reactants])
        if substant_in_previous:
          # end with reaction, need to cut off finally
          new_res_paths.append(path)
        else:
          new_res_paths.extend([path[:] + ([reactant]) for reactant in temp_reactants])
      else:
        temp_reactions = list(find_reactions_from_product(path[-1]))
        temp_wait_process_reactions = [reaction for reaction in temp_reactions if (reaction not in path and reaction not in duplicated_reactions)]
        # print("temp_reactions", temp_reactions, path[-1]) #################
        if not temp_wait_process_reactions:
          new_res_paths.append(path)
          continue
        new_res_paths.extend([path[:] + ([reaction]) for reaction in temp_wait_process_reactions])
        duplicated_reactions.extend(temp_wait_process_reactions)
    res_path = new_res_paths[:]
    new_hash = hash(str(res_path))
    # print(res_path, new_hash) #################
    if new_hash == result_hash:
      break
    else:
      result_hash = new_hash
  return res_path

In [ ]:
paths = search_one("carbon dioxide")

print("----------------- Result ------------------------")
for path in paths:
  print(path)